<a href="https://colab.research.google.com/github/sankeawthong/Project-1-Lita-Chatbot/blob/main/Hybrid%20LR-LSTM%20for%2010%20class%20classifications%20based%20on%20UNSW-NB15%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Hybrid LR-LSTM for 10 class classifications based on UNSW-NB15 dataset**

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Flatten

In [2]:
# Load dataset
dataset = pd.read_csv("dataset_P2.2.csv")
dataset = dataset.dropna() # Remove missing values
X = dataset.drop(['Class'], axis=1)
y = dataset['Class']

### **Data Preprocessing**

In [3]:
# Encode labels
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [4]:
# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
# Data preprocessing
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X, y = oversample.fit_resample(X, y)

In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### **Hybrid combination of LR and LSTM for 10 class classifications**

In [7]:
# Train logistic regression model
lr_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_acc = accuracy_score(y_test, lr_pred)
print("Logistic Regression Accuracy:", lr_acc)
print(classification_report(y_test, lr_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.5832589285714286
              precision    recall  f1-score   support

           0       0.94      0.86      0.90     11176
           1       0.52      0.29      0.37     11344
           2       0.28      0.48      0.35     11130
           3       0.28      0.21      0.24     11037
           4       0.67      0.37      0.48     11263
           5       0.71      0.66      0.68     11308
           6       0.97      0.96      0.96     11179
           7       0.38      0.33      0.35     11358
           8       0.52      0.83      0.64     11137
           9       0.74      0.85      0.79     11068

    accuracy                           0.58    112000
   macro avg       0.60      0.58      0.58    112000
weighted avg       0.60      0.58      0.58    112000



In [8]:
# Train LSTM model
num_classes = len(np.unique(y))
input_dim = X_train.shape[1]
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
lstm_model = Sequential()
lstm_model.add(LSTM(64, input_shape=(1, input_dim), activation='relu', return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(32, activation='relu'))
lstm_model.add(Dense(10, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
y_train_onehot = pd.get_dummies(y_train).values
lstm_model.fit(X_train, y_train_onehot, epochs=15, batch_size=32, verbose=0)
lstm_pred = lstm_model.predict(X_test)
lstm_pred = np.argmax(lstm_pred, axis=1)
lstm_acc = accuracy_score(y_test, lstm_pred)
print("LSTM Accuracy:", lstm_acc)
print(classification_report(y_test, lstm_pred))

3500/3500 [==============================] - 8s 2ms/step
LSTM Accuracy: 0.7360446428571429
              precision    recall  f1-score   support

           0       0.99      0.92      0.95     11176
           1       0.58      0.43      0.49     11344
           2       0.39      0.69      0.50     11130
           3       0.34      0.35      0.34     11037
           4       0.78      0.46      0.58     11263
           5       0.85      0.85      0.85     11308
           6       1.00      0.98      0.99     11179
           7       0.88      0.78      0.82     11358
           8       0.85      0.92      0.89     11137
           9       0.97      0.98      0.98     11068

    accuracy                           0.74    112000
   macro avg       0.76      0.74      0.74    112000
weighted avg       0.76      0.74      0.74    112000



In [9]:
# Combine predictions
lr_probs = lr_model.predict_proba(np.reshape(X_test, (X_test.shape[0], -1)))
lstm_probs = lstm_model.predict(X_test)
combined_probs = (lr_probs + lstm_probs) / 2
combined_pred = np.argmax(combined_probs, axis=1)
combined_acc = accuracy_score(y_test, combined_pred)
print("Combined Accuracy:", combined_acc)
print(classification_report(y_test, combined_pred))

3500/3500 [==============================] - 7s 2ms/step
Combined Accuracy: 0.7250892857142858
              precision    recall  f1-score   support

           0       0.99      0.92      0.95     11176
           1       0.55      0.44      0.49     11344
           2       0.38      0.68      0.49     11130
           3       0.33      0.28      0.30     11037
           4       0.79      0.43      0.56     11263
           5       0.84      0.85      0.84     11308
           6       0.99      0.98      0.98     11179
           7       0.82      0.78      0.80     11358
           8       0.82      0.93      0.87     11137
           9       0.92      0.98      0.95     11068

    accuracy                           0.73    112000
   macro avg       0.74      0.73      0.72    112000
weighted avg       0.74      0.73      0.72    112000

